# Main

In [ ]:
import os
import rasterio

In [ ]:
def get_images_path(path, satellite):
    '''
        Returns a list of paths of images and a list of names of zones. 
        Satellite must be 'sen1' or 'sen2'.
        Path must be the directory with 'sen1' and 'sen2' subdirectories inside.
    '''
    path = os.path.join(path, satellite)
    images_path = []
    zones = os.listdir(path)

    for zone in zones:
        zone_path = os.path.join(path, zone)
        images = os.listdir(zone_path)
        
        for image in images:
            image_path = os.path.join(zone_path, image)
            images_path.append(image_path)
            
    images_path.sort()
    
    return images_path, zones

In [ ]:
def split_S2_images(s2_paths):
    '''
        Splits the list of Sentinel-2 images in RGB images and cloud masks.
        The path of the images must contains 'RGB' or 'CM'.
    '''
    cloud_mask = []
    rgb = []
    
    for p in s2_paths:
        if "RGB" in p:
            rgb.append(p)
        elif "CM" in p:
            cloud_mask.append(p)
    
    return rgb, cloud_mask

In [ ]:
#Import the dataset 
dataset_path = '/content/drive/My Drive/timeseries_dataset'
s2_paths, s2_zones = get_images_path(dataset_path, 'sen2')
s1_images, s1_zones = get_images_path(dataset_path, 'sen1')
s2_images, cloud_masks = split_S2_images(s2_paths)

print(len(s2_images), len(s1_images))
print(len(s2_zones), len(s1_zones))

#Preprocess phase

In [ ]:
def get_s1_image(image_path):
    '''
        Returns an grayscale image (VV Sentinel-1) using a path.
        Image path must be the full path of an grayscale (VV) image (tif format).
    '''

    dataset = rasterio.open(image_path)
    if dataset.shape != (256, 256):
      vv = dataset.read(1, out_shape=(256, 256))
    else:
      vv = dataset.read(1)
    dataset.close()
    
    #vv = 10 * np.log10(vv)
    vv = np.clip(vv, -30, 15)

    #Normalization in [-1, 1]
    vv = 2 * ((vv - vv.min())/(vv.max() - vv.min())) - 1
    #Normalization in [0, 1]
    #vv = (vv - vv.min())/(vv.max() - vv.min())
    
    return vv.astype(float)

In [ ]:
def get_s2_image(image_path):
    '''
        Returns an RGB image using a path.
        Image path must be the full path of an RGB image (tif format).
    '''
    dataset = rasterio.open(image_path)
    if dataset.shape != (256, 256):
      r = dataset.read(1, out_shape=(256, 256))
      g = dataset.read(2, out_shape=(256, 256))
      b = dataset.read(3, out_shape=(256, 256))
    else:
      r = dataset.read(1)
      g = dataset.read(2)
      b = dataset.read(3)
    dataset.close()
    
    #RGB composite
    rgb = np.zeros((r.shape[0], r.shape[1], 3))
    rgb[..., 0] = r
    rgb[..., 1] = g
    rgb[..., 2] = b
    
    #Normalization in [-1, 1]
    rgb = 2 * ((rgb - rgb.min())/(rgb.max() - rgb.min())) - 1
    #Normalization in [0, 1]
    #rgb = (rgb - rgb.min())/(rgb.max() - rgb.min())
    
    return rgb.astype(float)

# Model&Train

In [ ]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.models import load_model
from numpy import zeros, ones
from numpy.random import randint
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.initializers import RandomNormal
from tensorflow.python.keras.models import Model, Input
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization
from matplotlib import pyplot
from skimage import metrics

In [ ]:
#Discriminator model
def define_discriminator(image_shape_sar, image_shape_opt):
  init = RandomNormal(stddev=0.02)
  in_src_image = Input(shape=image_shape_sar)
  in_target_image = Input(shape=image_shape_opt)
  merged = Concatenate()([in_src_image, in_target_image])
  
  d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged) 
  d = LeakyReLU(alpha=0.2)(d)
  
  d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  
  d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  
  d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  
  d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
  d = BatchNormalization()(d)
  d = LeakyReLU(alpha=0.2)(d)
  
  d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
  patch_out = Activation('sigmoid')(d)
  
  model = Model([in_src_image, in_target_image], patch_out)       
  opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
  
  return model

In [ ]:
#Generator model
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	init = RandomNormal(stddev=0.02)
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
 
	if batchnorm:
		g = BatchNormalization()(g, training=True)

	g = LeakyReLU(alpha=0.2)(g)
 
	return g
 

def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	init = RandomNormal(stddev=0.02)
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	g = BatchNormalization()(g, training=True)
	
	if dropout:
		g = Dropout(0.5)(g, training=True)
	
	g = Concatenate()([g, skip_in])
	g = Activation('relu')(g)
 
	return g
 
 
def define_generator(image_shape=(256,256,1)):
	
 init = RandomNormal(stddev=0.02)
 in_image = Input(shape=image_shape)

 e1 = define_encoder_block(in_image, 64, batchnorm=False)
 e2 = define_encoder_block(e1, 128)
 e3 = define_encoder_block(e2, 256)
 e4 = define_encoder_block(e3, 512)
 e5 = define_encoder_block(e4, 512)
 e6 = define_encoder_block(e5, 512)
 e7 = define_encoder_block(e6, 512)
	
 b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
 b = Activation('relu')(b)
	
 d1 = decoder_block(b, e7, 512)
 d2 = decoder_block(d1, e6, 512)
 d3 = decoder_block(d2, e5, 512)
 d4 = decoder_block(d3, e4, 512, dropout=False)
 d5 = decoder_block(d4, e3, 256, dropout=False)
 d6 = decoder_block(d5, e2, 128, dropout=False)
 d7 = decoder_block(d6, e1, 64, dropout=False)

 g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
 out_image = Activation('tanh')(g)

 model = Model(in_image, out_image)

 return model

In [ ]:
#GAN model
def define_gan(g_model, d_model, image_shape_sar):
	
 d_model.trainable = False
 in_src = Input(shape=image_shape_sar)
 
 gen_out = g_model(in_src)
 dis_out = d_model([in_src, gen_out])   

 model = Model(in_src, [dis_out, gen_out])   
 opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
 model.compile(loss=['binary_crossentropy', 'mse'], optimizer=opt, loss_weights=[1, 100])
	
 return model

In [ ]:
#Prepare data: the label y is 0 for fake samples and 1 for real samples
def select_batch(s1data, s2data, n_samples, patch_shape):
 X1=[]
 X2=[]
 ix = randint(0, 7983, n_samples)
 for elem in ix:
	 s1image = get_s1_image(s1data[elem])
	 s1image = np.reshape(s1image, (1,256,256,1))
	 s1image = despeckle_model.predict(s1image)
	 #X1.append(get_s1_image(s1data[elem]))
	 X1.append(s1image)
	 X2.append(get_s2_image(s2data[elem]))
 y = ones((n_samples, patch_shape, patch_shape, 1))
 return [X1, X2], y
 
def generate_fake_samples(g_model, samples, patch_shape):
	X = g_model.predict(samples)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y


#Save the model and plot results 
def summarize_performance(step, g_model, s1data, s2data, n_samples=3):
 #f = open("/content/drive/My Drive/ESA/GAN/file.txt", "a")
 #[X_realA, X_realB], _ = select_batch(s1data, s2data, n_samples, 1)
 #X_realA = np.array(X_realA).reshape(np.array(X_realA).shape[0], 256, 256, 1)
 #X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
 #X_realA = np.array(X_realA).reshape(np.array(X_realA).shape[0], 256, 256)
 
 #Scale all pixels from [-1,1] to [0,1]
 #X_realA = (np.array(X_realA) + 1) / 2.0
 #X_realB = (np.array(X_realB) + 1) / 2.0
 #X_fakeB = (np.array(X_fakeB) + 1) / 2.0

 '''
 for i in range(n_samples):
	 pyplot.subplot(3, n_samples, 1 + i)
	 pyplot.axis('off')
	 pyplot.imshow(X_realA[i])
	
 for i in range(n_samples):
	 pyplot.subplot(3, n_samples, 1 + n_samples + i)
	 pyplot.axis('off')
	 pyplot.imshow(X_fakeB[i])
	
 for i in range(n_samples):
	 pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
	 pyplot.axis('off')
	 pyplot.imshow(X_realB[i])
 '''
	 
	 #p = metrics.peak_signal_noise_ratio(X_realB[i], X_fakeB[i])
	 #f.write("Step " + str(step) + ": sample n° " + str(i) + " psnr value -> " + str(p) + "\n")
	 #s = metrics.structural_similarity(X_realB[i], X_fakeB[i], multichannel=True)
	 #f.write("Step " + str(step) + ": sample n° " + str(i) + " ssim value -> " + str(s) + "\n")
	
 #filename1 = 'plot_%06d.png' % (step+1)
 #pyplot.savefig('/content/drive/My Drive/ESA/GAN/' + filename1)
 #pyplot.close()

 filename2 = 'model_%06d.h5' % (step+1)
 g_model.save('/content/drive/My Drive/ESA/GAN/' + filename2)
 print('>Saved: %s' % (filename2))
 #print('>Saved: %s and %s' % (filename1, filename2))
 #f.close()

In [ ]:
#Train phase
def train(d_model, g_model, gan_model, s1data, s2data, n_epochs=300, n_batch=32):
	n_patch = d_model.output_shape[1]
	batch_per_epoch = int(len(s1data) / n_batch)
	n_steps = batch_per_epoch * n_epochs

	for i in range(n_steps):
		[X_realA, X_realB], y_real = select_batch(s1data, s2data, n_batch, n_patch)
		X_realA = np.array(X_realA).reshape(np.array(X_realA).shape[0], 256, 256, 1)
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)

		d_loss1 = d_model.train_on_batch([np.array(X_realA), np.array(X_realB)], y_real)
		d_loss2 = d_model.train_on_batch([np.array(X_realA), np.array(X_fakeB)], y_fake)
		g_loss, _, _ = gan_model.train_on_batch(np.array(X_realA), [y_real, np.array(X_realB)])

		print("Step: "+ str(i+1) +" -> d1_loss: " + str(d_loss1) + " | d2_loss: " + str(d_loss2) + " | g_loss: " + str(g_loss))
		if ((i+1) % 500 == 0) or (i == 0) or (i == n_steps-1): 
			summarize_performance(i, g_model, s1data, s2data)

In [ ]:
image_shape_sar = (256, 256, 1)
image_shape_opt = (256, 256, 3)

despeckle_model = load_model('/content/drive/My Drive/ESA/GAN/despeckling.h5')
discriminator_model = define_discriminator(image_shape_sar, image_shape_opt)
#generator_model = define_generator(image_shape_sar)
generator_model = load_model('/content/drive/My Drive/ESA/GAN/Weights/model_014000.h5')
gan_model = define_gan(generator_model, discriminator_model, image_shape_sar)

train(discriminator_model, generator_model, gan_model, s1_images, s2_images)